Imagine um chef que recebe uma pilha de ingredientes, como farinha, açúcar, carne moída, cebola, e especiarias. Ele não sabe o que fazer com eles, mas ao analisar as combinações, ele percebe que farinha, açúcar e ovos aparecem juntos com frequência, formando a "receita" de um bolo. Já carne moída e cebola formam outra "receita" de um prato salgado.

O LDA (Latent Dirichlet Allocation), no contexto de modelagem de tópicos, é como esse chef. Ele pega uma coleção de textos e, sem ler, descobre os tópicos ocultos ("as receitas") que os unem, analisando quais palavras tendem a aparecer juntas.

Neste script, vamos usar um pequeno conjunto de frases para ver o LDA encontrar os tópicos por trás delas.

In [1]:
!pip install gensim pyLDAvis

In [2]:
# =================================================================
# 1. Configuração e Preparação dos Dados
# =================================================================

# Importando as bibliotecas necessárias
import numpy as np
import pandas as pd
import nltk
from nltk.corpus import stopwords
from gensim.corpora.dictionary import Dictionary
from gensim.models.ldamodel import LdaModel
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
import warnings

# Baixando os dados de stopwords (palavras comuns como 'a', 'o', 'e')
nltk.download('stopwords')
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=UserWarning)

print("Passo 1: Definindo os textos para o nosso 'chef' analisar.")
print("-" * 50)

# Criando um conjunto de documentos de exemplo
documents = [
    "A galáxia de Andrômeda é nossa vizinha mais próxima.",
    "A Terra é o terceiro planeta do sol e abriga a vida.",
    "Astrônomos estudam estrelas e planetas usando telescópios.",
    "O bolo de chocolate é o meu favorito. A receita leva farinha, ovos e açúcar.",
    "Cozinhar massas e molhos é uma arte na culinária italiana.",
    "Eu adoro assar biscoitos com manteiga e essência de baunilha.",
    "Explorando a Lua, a NASA busca segredos sobre o nosso sistema solar."
]

# =================================================================
# 2. Pré-processamento dos Textos
# =================================================================

print("\nPasso 2: Processando os textos para remover palavras irrelevantes.")
print("-" * 50)

# Criando uma lista de palavras irrelevantes (stopwords) em português
stop_words = set(stopwords.words('portuguese'))

def preprocess_text(text):
    """Função para limpar e tokenizar o texto."""
    # Quebrando a frase em palavras (tokenização)
    words = text.lower().split()
    # Removendo stopwords e pontuação
    words = [word for word in words if word.isalpha() and word not in stop_words]
    return words

# Aplicando a função em todos os documentos
processed_docs = [preprocess_text(doc) for doc in documents]
print(processed_docs)

# =================================================================
# 3. Preparando os Dados para o LDA
# =================================================================

print("\nPasso 3: Criando o 'dicionário de ingredientes' e a 'lista de compras'.")
print("-" * 50)

# Criando um dicionário que mapeia cada palavra única a um ID
id2word = Dictionary(processed_docs)

# Criando o Corpus (Bag of Words). Para cada documento, é uma lista de (ID da palavra, contagem).
corpus = [id2word.doc2bow(doc) for doc in processed_docs]
print(corpus[0]) # Mostrando o primeiro documento como exemplo

# =================================================================
# 4. Treinando o Modelo LDA
# =================================================================

print("\nPasso 4: Treinando o nosso 'chef' (o modelo LDA) para encontrar 2 tópicos.")
print("-" * 50)

# Criando e treinando o modelo LDA
# num_topics: o número de tópicos (receitas) que esperamos encontrar
# id2word: o dicionário que criamos
# passes: quantas vezes o modelo vai revisar os dados
lda_model = LdaModel(corpus=corpus,
                     id2word=id2word,
                     num_topics=2,
                     random_state=42,
                     passes=10)

# =================================================================
# 5. Interpretando os Tópicos
# =================================================================

print("\nPasso 5: Revelando os tópicos encontrados pelo nosso 'chef'.")
print("-" * 50)

# Imprimindo os tópicos
topics = lda_model.print_topics(num_words=5) # Mostrando as 5 palavras mais relevantes para cada tópico
for topic in topics:
    print(topic)

print("\nAnalisando a saída:")
print("• O **Tópico 0** parece ser sobre **'Ciência e Espaço'**, com palavras como 'planetas', 'galáxia', 'terra'.")
print("• O **Tópico 1** parece ser sobre **'Comida e Culinária'**, com palavras como 'bolo', 'massa', 'cozinhar'.")

# =================================================================
# 6. Visualizando os Tópicos (O 'Mapa de Sabores')
# =================================================================

print("\nPasso 6: Gerando uma visualização interativa para explorar os tópicos.")
print("-" * 50)
print("Aguarde a visualização aparecer. Ela permite explorar as palavras de cada tópico.")

# Gerando a visualização interativa
vis = gensimvis.prepare(lda_model, corpus, id2word)

# Exibindo a visualização no Colab
pyLDAvis.display(vis)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Passo 1: Definindo os textos para o nosso 'chef' analisar.
--------------------------------------------------

Passo 2: Processando os textos para remover palavras irrelevantes.
--------------------------------------------------
[['galáxia', 'andrômeda', 'vizinha'], ['terra', 'terceiro', 'planeta', 'sol', 'abriga'], ['astrônomos', 'estudam', 'estrelas', 'planetas', 'usando'], ['bolo', 'chocolate', 'receita', 'leva', 'ovos'], ['cozinhar', 'massas', 'molhos', 'arte', 'culinária'], ['adoro', 'assar', 'biscoitos', 'manteiga', 'essência'], ['explorando', 'nasa', 'busca', 'segredos', 'sobre', 'sistema']]

Passo 3: Criando o 'dicionário de ingredientes' e a 'lista de compras'.
--------------------------------------------------
[(0, 1), (1, 1), (2, 1)]

Passo 4: Treinando o nosso 'chef' (o modelo LDA) para encontrar 2 tópicos.
--------------------------------------------------

Passo 5: Revelando os tópicos encontrados pelo nosso 'chef'.
--------------------------------------------------
(0, '

Como Interpretar a Visualização:

Círculos grandes: Cada círculo é um tópico. Quanto maior o círculo, mais prevalente é o tópico na sua coleção de textos.

Distância entre os círculos: Se os círculos estão muito distantes, os tópicos são bem diferentes. Se estão próximos, os tópicos compartilham muitas palavras ou conceitos.

Palavras na barra lateral: Ao clicar em um círculo (tópico), a barra lateral mostra as palavras mais relevantes para aquele tópico. Isso ajuda a entender do que se trata a "receita"